In [1]:
import pandas
import pymorphy2
from nltk.tokenize import RegexpTokenizer
morph = pymorphy2.MorphAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

In [2]:
df = pandas.read_csv("C:\\Users\\sante\\Desktop\\мой нлп - Лист1-2.csv", encoding="utf-8")

# тестовые данные
valid = df["title"].isin(['Интерстеллар', "Омерзительная восьмерка", "Тройной форсаж: Токийский дрифт"])
test = df[valid]
del test['title']

# удалили наши данные(оставили данные для обучения)
df = df.loc[df['title'] != "Интерстеллар"]
df = df.loc[df['title'] != "Омерзительная восьмерка"]
df = df.loc[df['title'] != "Тройной форсаж: Токийский дрифт"]
del df['title']

In [3]:
df.head

<bound method NDFrame.head of                                                   text  label
0    Дон Блут – один из самых известных анимационны...     -1
1    Отрицательных рецензий на этот мультфильм очен...     -1
2    \nЧестно признаюсь: в детстве я если и смотрел...     -1
3    \nВпервые я познакомилась с творением студии '...     -1
4    \nЯ узнала про мультфильм 'Анастасия' на продл...     -1
..                                                 ...    ...
585  Питер Джексон уже не тот...  Пишу как человек,...     -1
586  Ждали-ждали - дождались  Около двух лет назад ...     -1
587  Три часа скукоты  В том, что «Хоббит: Нежданно...     -1
588  что-то случилось с любимым нами Средиземьем  В...     -1
589  Детям фильм смотреть нельзя  Фильм не понравил...     -1

[590 rows x 2 columns]>

In [4]:
def tokenizeWord(word):
    word = tokenizer.tokenize(word)
    if len(word) == 1:
        word = word[0]
    else:
        word = ""
    return word

In [5]:
uniqWords = []
tupleIndex = 0
sentences = []
for tuple in df.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    sentences.append(wordsList)
tupleIndex = 0
testSentences = []
for tuple in test.values:
    tupleIndex+=1
    words = tuple[0].split()
    wordsList = []
    for word in words:
        wordNF = tokenizeWord(morph.parse(word)[0].normal_form)
        uniqWords.append(wordNF)
        wordsList.append(wordNF)
    testSentences.append(wordsList)

In [6]:
import multiprocessing
from gensim.models import Word2Vec
uniqWords = set(uniqWords)
model = Word2Vec(sentences=sentences,min_count=1,size=50)

In [7]:
print(model.wv.most_similar('конец'))
print(model.wv.most_similar('фильм'))
print(model.wv.most_similar('он'))
print(model.wv.most_similar('режиссер'))
print(model.wv.most_similar('герой'))

[('амели', 0.9998201131820679), ('идея', 0.9998074769973755), ('тут', 0.9998012781143188), ('начинать', 0.9997977018356323), ('смысл', 0.9997847676277161), ('дело', 0.9997847080230713), ('здесь', 0.9997807145118713), ('начать', 0.9997804164886475), ('время', 0.9997758865356445), ('почему', 0.9997738599777222)]
[('посмотреть', 0.9989514946937561), ('ни', 0.9988264441490173), ('плохой', 0.9985913634300232), ('только', 0.9985641837120056), ('поход', 0.9985373020172119), ('кино', 0.9985154867172241), ('кто', 0.9984852075576782), ('несмотря', 0.9984041452407837), ('нужно', 0.9983731508255005), ('этот', 0.9983561635017395)]
[('нет', 0.9997385144233704), ('много', 0.9997354745864868), ('даже', 0.9997287392616272), ('мы', 0.9997202754020691), ('но', 0.9996963739395142), ('ты', 0.9996821880340576), ('ну', 0.9996635913848877), ('вот', 0.999660849571228), ('уже', 0.9996534585952759), ('все', 0.999649703502655)]
[('собственный', 0.9992032051086426), ('джон', 0.999193549156189), ('работа', 0.999190

In [8]:
def getMiddleValue(arrays):
    finalArray = []
    for i in range(50):
        finalArray.append(int(0))
    for array in arrays:
        for j in range(len(array)):
            finalArray[j] += array[j]
    for value in finalArray:
        value = value/len(arrays)
    return list(finalArray)
print(model.wv['блут'])

[ 0.00655722 -0.00448587  0.00135379 -0.00653863 -0.00624461  0.00416122
  0.01504749  0.03680665 -0.06079569  0.02115461 -0.07996404  0.02717822
 -0.04134552  0.05607507 -0.01689739  0.00066547  0.0287894   0.0324901
 -0.03891056  0.04444839  0.01099535 -0.00237673  0.03054265 -0.05335897
  0.01247396  0.00650215 -0.01516874  0.00621232  0.03730186 -0.00979579
 -0.04780384  0.02349715  0.00338331  0.00325101 -0.04447849 -0.04639287
 -0.04266626 -0.02022582  0.00584296  0.01900918  0.04012205 -0.03678497
  0.04464997  0.04940165  0.02589019  0.00507945  0.01536379  0.02687091
 -0.02062068 -0.03230522]


In [9]:
columns = []
for i in range(50):
    columns.append(i)
X_train = pandas.DataFrame(columns = columns)
index = 0
for comment in sentences:
    index += 1
    arrays = []
    for word in comment:
        vector = model.wv[word]
        arrays.append(vector)
    X_train.loc[index] = getMiddleValue(arrays)


In [10]:
Y_train = df["label"]
Y_test = test["label"]

In [11]:
from sklearn.ensemble import RandomForestClassifier
randomForestCLF = RandomForestClassifier(max_depth=20, random_state=0).fit(X_train, Y_train)

In [12]:
columns = []
for i in range(50):
    columns.append(i)
X_test = pandas.DataFrame(columns = columns)
index = 0
for comment in testSentences:
    index += 1
    arrays = []
    for word in comment:
        if word in list(model.wv.vocab.keys()):
            vector = model.wv[word]
            arrays.append(vector)
        else:
            vector = []
            for i in range(50):
                vector.append(0)
            arrays.append(vector)
    X_test.loc[index] = list(getMiddleValue(arrays))

In [13]:
from sklearn.metrics import precision_recall_fscore_support
randomForestPredict = randomForestCLF.predict(X_test)
randomForestMetric = precision_recall_fscore_support(test["label"].values, randomForestPredict, average='weighted')

In [14]:
print(randomForestMetric)

(0.47367488931056295, 0.4777777777777778, 0.4738387978142076, None)
